In [ ]:
import torch

rand_tensor = torch.rand(12000, 4, 32, 32)

In [ ]:
# print the size of the tensor in gigabytes
print(rand_tensor.element_size() * rand_tensor.nelement() / 1024**3)

In [ ]:
import trimesh

scene = trimesh.Scene()

In [ ]:
template_3dc = trimesh.load_mesh('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/3DC/template.ply')
template_3dc.export('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/template.off')

In [ ]:
import torch

template_original = trimesh.load_mesh('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/template.off')

verts_orig = torch.tensor(template_original.vertices, dtype=torch.float32)
faces_orig = torch.tensor(template_original.faces, dtype=torch.int32)

scene.geometry.clear()

scene.add_geometry(trimesh.creation.axis(origin_size=0.1))
scene.add_geometry(template_original)

scene.show()

In [ ]:
import numpy as np

corr_orig = torch.tensor(list(range(len(verts_orig)))) + 1
# save corr_orig as a .txt file to /home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original
np.savetxt('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/corr.txt', corr_orig.numpy(), fmt='%d')

In [ ]:
# read /home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/corr_orig.txt as torch tensor
corr_orig = torch.tensor(np.loadtxt('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/corr_orig.txt'), dtype=torch.int32)
corr_orig -= 1
corr_orig

# Remesh

In [ ]:
import my_code.sign_canonicalization.remesh as remesh
import utils.fmap_util as fmap_util
import torch

verts_r, faces_r = remesh.remesh_simplify_iso(
    verts=template_original.vertices,
    faces=template_original.faces,
    n_remesh_iters=10,
    remesh_targetlen=1,
    simplify_strength=1,
    )
corr_r = fmap_util.nn_query(
    verts_r,
    verts_orig, 
    )

template_remeshed = trimesh.Trimesh(
    vertices=verts_r,
    faces=faces_r
    )

scene.geometry.clear()
scene.add_geometry(trimesh.creation.axis(origin_size=0.1))

# scene.add_geometry(template_original)
scene.add_geometry(template_remeshed)

scene.show()

In [ ]:
# save the template to /home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed, along with the correspondence
template_remeshed.export('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/template.off')
np.savetxt('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/corr.txt', corr_r.numpy() + 1, fmt='%d')


In [ ]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()
plotting_utils.plot_p2p_map(
    scene,
    verts_r, faces_r,
    verts_orig, faces_orig,
    corr_r,
    )
scene.show()
    

# Remesh and simplify

In [ ]:
import my_code.sign_canonicalization.remesh as remesh
import utils.fmap_util as fmap_util
import torch

verts_rs, faces_rs = remesh.remesh_simplify_iso(
    verts=verts_orig,
    faces=faces_orig,
    n_remesh_iters=10,
    remesh_targetlen=1,
    simplify_strength=0.4,
    )
corr_rs = fmap_util.nn_query(
    verts_rs,
    verts_orig, 
    )

template_remeshed_simpl = trimesh.Trimesh(
    vertices=verts_rs,
    faces=faces_rs
    )

scene.geometry.clear()
scene.add_geometry(trimesh.creation.axis(origin_size=0.1))

# scene.add_geometry(template_original)
# scene.add_geometry(template_remeshed)
scene.add_geometry(template_remeshed_simpl)

scene.show()

In [ ]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()
plotting_utils.plot_p2p_map(
    scene,
    verts_rs, faces_rs,
    verts_orig, faces_orig,
    corr_rs,
    )
scene.show()
    

In [ ]:
template_remeshed_simpl.export(
    '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_simplified/template.off'
    )
np.savetxt(
    '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_simplified/corr.txt',
    corr_rs.numpy() + 1, fmt='%d')


# Remesh and Smooth

In [ ]:
verts_r_smoothed, faces_r_smoothed = remesh.remesh_simplify_iso(
    verts=verts_orig,
    faces=faces_orig,
    n_remesh_iters=10,
    remesh_targetlen=1,
    simplify_strength=1,
    )
template_r_smoothed = trimesh.Trimesh(
    vertices=verts_r_smoothed,
    faces=faces_r_smoothed
    )
trimesh.smoothing.filter_laplacian(template_r_smoothed, lamb=0.5, iterations=3)
# trimesh.smoothing.filter_taubin(template_r_smoothed, lamb=0.5, iterations=5)

verts_r_smoothed = torch.tensor(template_r_smoothed.vertices, dtype=torch.float32)
faces_r_smoothed = torch.tensor(template_r_smoothed.faces, dtype=torch.int32)

corr_r_smoothed = fmap_util.nn_query(
    verts_r_smoothed,
    verts_orig, 
    )

scene.geometry.clear()
scene.add_geometry(trimesh.creation.axis(origin_size=0.1))

# scene.add_geometry(template_original)
# scene.add_geometry(template_remeshed)
scene.add_geometry(template_r_smoothed)

scene.show()

In [ ]:
scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    verts_r_smoothed, faces_r_smoothed,
    verts_orig, faces_orig,
    corr_r_smoothed,
    )
scene.show()

In [ ]:
template_r_smoothed.export(
    '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_smoothed/template.off'
    )
np.savetxt(
    '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_smoothed/corr.txt',
    corr_r_smoothed.numpy() + 1, fmt='%d')


# Test all templates

In [ ]:
import trimesh

scene = trimesh.Scene()

In [ ]:
mesh_3dc = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/3DC/template.ply'

mesh_orig = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/template.off'
corr_orig = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/original/corr.txt'

mesh_remeshed = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/template.off'
corr_remeshed = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed/corr.txt'

mesh_remeshed_simplified = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_simplified/template.off'
corr_remeshed_simplified = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_simplified/corr.txt'

mesh_remeshed_smoothed = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_smoothed/template.off'
corr_remeshed_smoothed = '/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/remeshed_smoothed/corr.txt'

In [ ]:
import torch
import numpy as np
import my_code.utils.plotting_utils as plotting_utils

mesh_3dc = trimesh.load_mesh(mesh_3dc)
verts_3dc = torch.tensor(mesh_3dc.vertices, dtype=torch.float32)
faces_3dc = torch.tensor(mesh_3dc.faces, dtype=torch.int32)

def check_p2p(mesh_path, corr_path):
    scene.geometry.clear()
    scene.add_geometry(trimesh.creation.axis(origin_size=0.1))
    
    mesh_i = trimesh.load_mesh(mesh_path)
    
    verts_i = torch.tensor(mesh_i.vertices, dtype=torch.float32)
    faces_i = torch.tensor(mesh_i.faces, dtype=torch.int32)
    
    corr_i = torch.tensor(np.loadtxt(corr_path), dtype=torch.int32) - 1

    plotting_utils.plot_p2p_map(
        scene,
        verts_i, faces_i,
        verts_3dc, faces_3dc,
        corr_i,
        )
    # scene.show()

In [ ]:
# check_p2p(mesh_orig, corr_orig)
# check_p2p(mesh_remeshed, corr_remeshed)
# check_p2p(mesh_remeshed_simplified, corr_remeshed_simplified)
check_p2p(mesh_remeshed_smoothed, corr_remeshed_smoothed)
scene.show()

# Check the dataset

In [ ]:
from my_code.datasets.surreal_dataset_3dc import TemplateSurrealDataset3DC

augmentations = {
    "remesh": {
        "isotropic": {
            "n_remesh_iters": 10,
            "remesh_targetlen": 1,
            "simplify_strength_min": 0.2,
            "simplify_strength_max": 0.8,
        },
        "anisotropic": {
            "probability": 0.35,
                
            "n_remesh_iters": 10,
            "fraction_to_simplify_min": 0.2,
            "fraction_to_simplify_max": 0.6,
            "simplify_strength_min": 0.2,
            "simplify_strength_max": 0.5,
            "weighted_by": "face_count",
        },
    },
}

template_type = 'remeshed_smoothed'

dataset = TemplateSurrealDataset3DC(
    # shape_path=f'/home/s94zalek_hpc/3D-CODED/data/mmap_datas_surreal_train.pth',
    shape_path='/lustre/mlnvme/data/s94zalek_hpc-shape_matching/mmap_datas_surreal_train.pth',
    num_evecs=128,
    cache_lb_dir=None,
    return_evecs=True,
    mmap=True,
    augmentations=augmentations,
    template_path=f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/{template_type}/template.off',
    template_corr=np.loadtxt(
        f'/home/s94zalek_hpc/shape_matching/data/SURREAL_full/template/{template_type}/corr.txt',
        dtype=np.int32) - 1
)  

In [ ]:
data_i = dataset[200004]

scene.geometry.clear()

mesh_1 = trimesh.Trimesh(vertices=data_i['first']['verts'], faces=data_i['first']['faces'])
mesh_2 = trimesh.Trimesh(vertices=data_i['second']['verts'] + torch.tensor([1, 0, 0]), faces=data_i['second']['faces'])

scene.add_geometry(trimesh.creation.axis(origin_size=0.1))
scene.add_geometry(mesh_1)
scene.add_geometry(mesh_2)

scene.show()

In [ ]:
import matplotlib.pyplot as plt

l = 0
h = 32

fig, axs = plt.subplots(1, 2, figsize=(8, 5))

plotting_utils.plot_Cxy(fig, axs[0], data_i['second']['C_gt_xy'],
                        'xy', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[1], data_i['second']['C_gt_yx'],
                        'yx', l, h, show_grid=False, show_colorbar=False)
plt.show()

In [ ]:
import utils.fmap_util as fmap_util

C_gt_xy = data_i['second']['C_gt_xy']

p2p = fmap_util.fmap2pointmap(
    C12=C_gt_xy,
    evecs_x=data_i['first']['evecs'],
    evecs_y=data_i['second']['evecs'],
)

scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    data_i['first']['verts'], data_i['first']['faces'],
    data_i['second']['verts'], data_i['second']['faces'],
    p2p,
    )
scene.show()

In [ ]:
import torch

cached_C = torch.load(
    '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SURREAL/train/test_pt/train/C_gt_xy_0_2.pt'
)
cached_C.shape

In [ ]:


import matplotlib.pyplot as plt

l = 0
h = 32

fig, axs = plt.subplots(1, 2, figsize=(8, 5))

plotting_utils.plot_Cxy(fig, axs[0], cached_C[0],
                        'xy', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[1], cached_C[1],
                        'yx', l, h, show_grid=False, show_colorbar=False)
plt.show()